In [1]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [2]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [3]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,PROFILAXIS. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,40000,40000,100%,100.00%,40000,0.0,17/09/2018,11/09/2018
1,BLANQUEAMIENTO DENTAL 1. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,412500,412500,100%,100.00%,412500,0.0,17/09/2018,11/09/2018
2,CARILLA ESTETICA EN RESINA. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,17/09/2018,Sin Cita Producida
3,CARILLA ESTETICA EN RESINA. Pieza: 21,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,17/09/2018,Sin Cita Producida
4,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 22,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,116500,0,0.00%,0.00%,0,116500.0,17/09/2018,Sin Cita Producida
5,PLACA NEUROMIORELAJANTE ACTIVADA. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,299662,0,0.00%,0.00%,0,299700.0,17/09/2018,Sin Cita Producida
6,PROVISIONAL ESTETICO ATORNILLADO CON PERFIL DE EMERGENCIA.,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,REHABILITACIÓN,ALISSON SARAY MORA CASTAÑEDA,206700,0,0.00%,0.00%,0,206700.0,14/09/2018,Sin Cita Producida
7,CITA TALLADO SELECTIVO X PIEZA. Pieza: 21,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,OCLUSION Y DISFUNCION TEMPOROMANDIBULAR,ALISSON SARAY MORA CASTAÑEDA,118720,0,0.00%,0.00%,0,118700.0,14/09/2018,Sin Cita Producida
8,CORONA SOBRE IMPLANTE ZIRCONIO. Pieza: 22,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,REHABILITACIÓN,ALISSON SARAY MORA CASTAÑEDA,1478700,0,0.00%,0.00%,0,1478700.0,14/09/2018,Sin Cita Producida
9,PLACA NEUROMIORELAJANTE ACTIVADA. Pieza: 11,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,299662,0,0.00%,0.00%,0,299700.0,14/09/2018,Sin Cita Producida


In [4]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 437
REGISTROS PPE: 501


In [5]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

,Servicio,Clínica,Fecha Pres._x,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor_x,Especialidad,Explicado_x,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod,Fecha Pres._y,Cantidad Presupuestada,Doctor_y,Explicado_y,ttmto_homol,pieza,fecha_inicio
Semana,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,PROFILAXIS. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,40000,40000,100%,100.00%,40000,0.0,17/09/2018,11/09/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,BLANQUEAMIENTO DENTAL 1. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,412500,412500,100%,100.00%,412500,0.0,17/09/2018,11/09/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CARILLA ESTETICA EN RESINA. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,17/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CARILLA ESTETICA EN RESINA. Pieza: 21,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,17/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR. Pieza: 22,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,116500,0,0.00%,0.00%,0,116500.0,17/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PLACA NEUROMIORELAJANTE ACTIVADA. Pieza: 11,Dentix Calle 90,11/09/2018,BG056618,Nº 4390 - LUZ ANGELA ECHEVERY,0,3108628251,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,299662,0,0.00%,0.00%,0,299700.0,17/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PROVISIONAL ESTETICO ATORNILLADO CON PERFIL DE EMERGENCIA.,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,REHABILITACIÓN,ALISSON SARAY MORA CASTAÑEDA,206700,0,0.00%,0.00%,0,206700.0,14/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CITA TALLADO SELECTIVO X PIEZA. Pieza: 21,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,OCLUSION Y DISFUNCION TEMPOROMANDIBULAR,ALISSON SARAY MORA CASTAÑEDA,118720,0,0.00%,0.00%,0,118700.0,14/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CORONA SOBRE IMPLANTE ZIRCONIO. Pieza: 22,Dentix Calle 90,11/09/2018,BG058157,Nº 4391 - JEANNETE RAMOS NIVIA,0,3112766076,DIEGO ARMANDO LOPEZ MENDEZ,REHABILITACIÓN,ALISSON SARAY MORA CASTAÑEDA,1478700,0,0.00%,0.00%,0,1478700.0,14/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes

Numero de tratamientos faltantes por crono: 110


Fecha Pres._x  \
Clínica         Nº Historia Servicio                                                                               
Dentix Calle 90 BG056618    PROFILAXIS.   Pieza: 11                                                 11/09/2018     
                            BLANQUEAMIENTO DENTAL 1.   Pieza: 11                                    11/09/2018     
                            CARILLA ESTETICA EN RESINA.   Pieza: 11                                 11/09/2018     
                            CARILLA ESTETICA EN RESINA.   Pieza: 21                                 11/09/2018     
                            RESINA FOTOPOLIMERIZADO 3 SUP POSTERIOR O POSTERIOR.   Pieza: 22        11/09/2018     
                            PLACA NEUROMIORELAJANTE ACTIVADA.   Pieza: 11                           11/09/2018     
                BG058157    PROVISIONAL ESTETICO ATORNILLADO CON PERFIL DE EMERGENCIA.              11/09/2018     
                            CITA TALLADO SELECTIVO X PIEZA.   Pieza: 21                             11/09/2018     
                            CORONA SOBRE IMPLANTE ZIRCONIO.   Pieza: 22                             11/09/2018     
                            PLACA NEUROMIORELAJANTE ACTIVADA.   Pieza: 11                           11/09/2018     
                BG057617    PROTESIS TEMPORAL  4/8 DIENTES .   Pieza: 11                            11/09/2018     
                BG053926    REGENERACION OSEA (SIN HUESO NI MEMBRANA).                              11/09/2018     
                            ELEVACION AL SENO ATRAUMATICA (POR CUADRANTE).   Pieza: 24              11/09/2018     
                BG053075    DETARTRAJE POR ARCADA.                                                  11/09/2018     
                            PROFILAXIS.   Pieza: 11                                                 11/09/2018     
                BG058172    PROVISIONAL ESTETICO ATORNILLADO CON PERFIL DE EMERGENCIA.              11/09/2018     
                            IMPLANTE.   Pieza: 25                                                   11/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 44                                      11/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 45                                      11/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 46                                      11/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 25                                      11/09/2018     
                BG058176    PROFILAXIS.   Pieza: 11                                                 11/09/2018     
                            DETARTRAJE POR ARCADA.                                                  11/09/2018     
                            ENTRADA BRACKETS ARCADA INFERIOR AUTOLIGABLES ESTETICOS.                11/09/2018     
                            ENTRADA BRACKETS ARCADA SUPERIOR AUTOLIGABLES ESTETICOS.                11/09/2018     
                            MENSUALIDAD ORTODONCIA AUTOLIGABLES ESTETICOS.                          11/09/2018     
                            ESTUDIO CEFALOMETRICO.   Pieza: 11                                      11/09/2018     
                            RETENEDOR FIJO 3X3 EN ALAMBRE.   Pieza: 11                              11/09/2018     
                            EXODONCIA QUIRURGICA DE CORDAL (INCLUIDO SEMIERUPCIONADO).   Pieza: 38  11/09/2018     
                            EXODONCIA QUIRURGICA DE CORDAL (INCLUIDO SEMIERUPCIONADO).   Pieza: 48  11/09/2018     
                BG057933    PROVISIONALES.   Pieza: 24                                              10/09/2018     
                BG057717    RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 46         10/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 47         10/09/2018     
                            INCRUSTACIO